In [ ]:
#pip install nibabel

In [1]:
import nibabel as nib
import numpy as np
import os
import pandas as pd
from scipy.stats import skew, kurtosis
from skimage import measure
from tqdm import tqdm

In [2]:
#This code locates and loads MRI and segmentation data from a nested directory structure. 
#It searches for patient folders within a root directory, identifying them by specific naming conventions. 
#For each patient, it loads corresponding NIfTI image files using the nibabel library, 
#   storing the image data in dictionaries keyed by patient ID. 
#Error handling is included to manage missing or corrupt files, and a progress bar displays the loading process.

rootDir = r"C:\Users\adity\Documents\Python projects\Rembrandt\Rembrandt_data\Rembrandt_Final_Data"

patient_dirs = []
for patients_folder in os.listdir(rootDir):
    if patients_folder.startswith("Patients_") and os.path.isdir(os.path.join(rootDir, patients_folder)):
        patients_folder_path = os.path.join(rootDir, patients_folder)

        for patient_folder in os.listdir(patients_folder_path):
            if patient_folder.startswith(("HF", "900")) and os.path.isdir(os.path.join(patients_folder_path, patient_folder)):
                patient_folder_path = os.path.join(patients_folder_path, patient_folder)
                patient_dirs.append(patient_folder_path)

mri_data = {}
segmentation_data = {}

for patient_dir in tqdm(patient_dirs, desc="Loading Data"):
    patient_id = os.path.basename(patient_dir)
    mri_path = None
    segmentation_path = None
    for filename in os.listdir(patient_dir):
        if filename.endswith("t1_LPS_rSRI.nii.gz") or filename.endswith("t1_LPS_rSRI.nii"):
            mri_path = os.path.join(patient_dir, filename)
        elif filename.endswith("GlistrBoost_out.nii.gz"):
            segmentation_path = os.path.join(patient_dir, filename)

    if mri_path and segmentation_path:
        try:
            mri_img = nib.load(mri_path)
            segmentation_img = nib.load(segmentation_path)
            mri_data[patient_id] = mri_img.get_fdata()
            segmentation_data[patient_id] = segmentation_img.get_fdata()
        except Exception as e:
            print(f"Error loading files for {patient_id}: {e}")
    else:
        print(f"Warning: MRI or segmentation file missing for {patient_id}")

Loading Data: 100%|██████████| 64/64 [00:16<00:00,  3.84it/s]


In [3]:
#This code defines a function extract_simple_features that calculates basic statistical 
#   and morphological features from an MRI image and its corresponding segmentation mask, 
#   focusing on the tumor region.

#First, it creates a binary mask indicating the tumor region based on the segmentation data. 
#Then, it extracts the MRI intensities within this tumor region. 
#It calculates the tumor volume by summing the mask values. 
#If a tumor region exists, it calculates the mean, standard 
#   deviation, skewness, and kurtosis of the MRI intensities within the tumor. 
#It also attempts to calculate the tumor's surface area using marching cubes, 
#   handling potential errors. Finally, it calculates the dimensions of the tumor's bounding box. 
#If no tumor region is found, it sets all features to NaN. 
#The function returns a dictionary containing these calculated features.

def extract_simple_features(mri, segmentation):
    """Extracts simple statistical and morphological features."""

    mask = segmentation > 0
    tumor_region = mri[mask]

    features = {}
    features["volume"] = np.sum(mask)

    if len(tumor_region) > 0:
        features["mean_intensity"] = np.mean(tumor_region)
        features["std_intensity"] = np.std(tumor_region)
        features["skewness"] = skew(tumor_region)
        features["kurtosis"] = kurtosis(tumor_region)

        try:
            vertices, faces, _, _ = measure.marching_cubes(segmentation, level = 1.75) #level may need adjusting.
            if len(faces) > 0:
                features["surface_area"] = measure.mesh_surface_area(vertices, faces)
            else:
                features["surface_area"] = 1
        except:
            features["surface_area"] = np.nan

        coords = np.argwhere(mask)
        x_min, y_min, z_min = coords.min(axis = 0)
        x_max, y_max, z_max = coords.max(axis = 0)
        features["bounding_box_x"] = x_max - x_min
        features["bounding_box_y"] = y_max - y_min
        features["bounding_box_z"] = z_max - z_min

    else:
        features["mean_intensity"] = np.nan
        features["std_intensity"] = np.nan
        features["skewness"] = np.nan
        features["kurtosis"] = np.nan
        features["surface_area"] = np.nan
        features["bounding_box_x"] = np.nan
        features["bounding_box_y"] = np.nan
        features["bounding_box_z"] = np.nan

    return features

In [4]:
all_features = []

for patient_id in tqdm(mri_data.keys(), desc="Extracting Features"):
    features = extract_simple_features(mri_data[patient_id], segmentation_data[patient_id])
    features["patient_id"] = patient_id
    all_features.append(features)

Extracting Features: 100%|██████████| 64/64 [00:24<00:00,  2.66it/s]


In [5]:
feature_matrix = pd.DataFrame(all_features)
feature_matrix.to_csv("morphological.csv", index=False)
print("Simple features extracted and saved to morphological.csv")

Simple features extracted and saved to morphological.csv
